# Tutorial: Simulation

# Part 1: Algorithmic Exercises on Solvers and Numerical Methods

## Non-linear solver exercises

These exercises are designed to reinforce the concepts covered in Chapter 4, including linear and nonlinear solvers, conditioning, scaling, Jacobians, Hessians, and Newton methods.

### Exercise 1: Computing Jacobians and Hessians Analytically and Numerically
Objective: Practice deriving and approximating Jacobians and Hessians for nonlinear functions.


Start with the scalar function:

$$f(x_1, x_2) = x_1^2 \sin(x_2) + e^{x_1 x_2}$$

1. Use SymPy to compute the analytical Jacobian (gradient) and Hessian at $(x_1, x_2) = (1, \pi/2)$.
2. Approximate the Jacobian and Hessian numerically using finite differences (e.g., with scipy.optimize.approx_fprime for gradient, and a similar approach for Hessian).
3. Compare the results and discuss truncation vs. round-off errors in finite differences.

Hints: For numerical Hessian, finite-difference the gradient. Vary the step size $\epsilon$ (e.g., 1e-6 to 1e-8) to observe error trade-offs.

The code below is a starting point with the symbolic and numerical coded in.



In [2]:
import sympy as sp
from scipy.optimize import approx_fprime
import numpy as np

# Analytical with SymPy
x1, x2 = sp.symbols('x1 x2')
f_analytical = x1**2 * sp.sin(x2) + sp.exp(x1 * x2)

# Numerical function
def f_num(x):
    return x[0]**2 * np.sin(x[1]) + np.exp(x[0] * x[1])

#
print(f"Expected result = [9.5562802  4.81047738]")

Analytical Jacobian: [[9.5562802  4.81047738]]
Numerical Jacobian: [9.55628713 4.81047929]


### Excercise 2: Vector-valued functions

Next, repeat the exercise for the vector function:

$$F(x_1, x_2) = [ x_1^2 \sin(x_2) + e^{x_1 x_2}, x_2^2 \sin(x_1) + e^{x_1 x_2}] $$

In [ ]:
# Analytical with SymPy
x1, x2 = sp.symbols('x1 x2')
f_analytical = [x1**2 * sp.sin(x2) + sp.exp(x1 * x2),
                x2**2 * sp.sin(x1) + sp.exp(x1 * x2)]

# Numerical function
def f_num(x):
    return [x[0]**2 * np.sin(x[1]) + np.exp(x[0] * x[1]),
            x[1]**2 * np.sin(x[0]) + np.exp(x[0] * x[1])]

### Exercise 3: Implementing a Single Gauss-Newton Iteration
#### 3.1) Gradient descent step

The purpose of this exercise is to gain intuition for the Gauss-Newton method through manual implementation and visualization.

For the nonlinear system $\mathbf{F}(\mathbf{x}) = \begin{pmatrix} x_1 + x_2 - 3 \\ x_1^2 + x_2^2 - 5 \end{pmatrix} = \mathbf{0}$:

1. Start from $\mathbf{x}_0 = (1, 1.5)$.
2. Compute the Jacobian $\mathbf{J}(\mathbf{x}_0)$ analytically or numerically.
3. Solve for the search direction $\mathbf{p}$ using the Gauss-Newton normal equations.
4. Update $\mathbf{x}_1 = \mathbf{x}_0 + \mathbf{p}$ and evaluate $\|\mathbf{F}(\mathbf{x}_1)\|$.
5. Visualize the contours of $S(\mathbf{x}) = \frac{1}{2} \|\mathbf{F}(\mathbf{x})\|^2$ and plot the step.

Hints: Use Matplotlib for contour plots. Compare with scipy.optimize.least_squares.

In [ ]:
# Definition of F(x) = 0:
def F(x):
    return np.array([x[0] + x[1] - 3,
                     x[0]**2 + x[1]**2 - 5])

#### 3.2) Improving Gauss-Newton step with line search

The above equation assumed that $\alpha_k = 1$ in the line search:

$$\mathbf{x_{k+1}} = \mathbf{x_k} + \alpha_k \mathbf{p_k}$$

When we have knowledge of the Hessian $f''(x) = \nabla^2 f(x) = H_f(x)\in \mathbb{R}^{d\times d}$, we can obtain the classical Newton step ($iff$ the Hessian is invertible)):

$$x_{k+1} = x_k - [f''(x_k)]^{-1} f'(x_k)$$

An operator that is closely related to the Hessian is the Laplacian operator, defined as the divergence of the gradient of a function. In multiple dimensions, the Laplacian is given by:

$$ \nabla \cdot \nabla  f = \sum_{i=1}^{d} \frac{\partial^2 f}{\partial x_i^2} $$

Because the Laplacian doesn't need to be inverted, it can be used to approximate the Newton step and is more suitable to higher dimensional problems with a large number of entries in the Hessian matrix. Implement the following two steps to improve the Gauss-Newton method:

##### 3.2.1) Pseudo-Newton step

Use the Laplacian operator to approximate value of $\alpha_k$.



##### 3.2.2) Full Newton step

Implement the full matrix multiplication to $H^-1 \nabla f$ to compute the Newton step.

##### 3.2.3) Compare the two methods

Compare the performance of 3.2.1 and 3.2.2 on a contour plot, a good metric for this performance is to measure the reduction in the residual $\|\mathbf{F}(\mathbf{x})\|$ after one iteration.

### 3.3) A full solver

You now have the building blocks to implement a full Gauss-Newton solver. Implement the iterative procedure:
1. Initialize $\mathbf{x}_0$.
2. While not converged:
   - Compute $\mathbf{F}(\mathbf{x}_k)$ and $\mathbf{J}(\mathbf{x}_k)$.
   - Solve for $\mathbf{p}_k$.
   - Perform line search to find $\alpha_k$.
   - Update $\mathbf{x}_{k+1} = \mathbf{x}_k + \alpha_k \mathbf{p}_k$.


Choose any of the methods you developed in 3.2 and compare the convergence behavior (number of iterations, final residual) with scipy.optimize.fsolve using the same initial guess.

Below is a skeleton to get you started, but feel free to modify as needed.

In [ ]:
def solve_gauss_newton(F, x0, tol=1e-6, max_iter=100):
    xk = x0
    for k in range(max_iter):
        # Compute F and J
        # Solve for pk
        #
        # Update x_k+1
        xk = xk + alpha_k * pk
        if np.linalg.norm(Fk) < tol:
            break
    return xk

---
## Linear Solver Exercises



## Exercise 4: Analyzing Matrix Conditioning and Scaling

The purpose of this exercise is to understand the impact of matrix conditioning on solution stability and explore scaling techniques to improve numerical accuracy.

Problem: Consider the linear system $\mathbf{Ax} = \mathbf{b}$ where:
$$\mathbf{A} = \begin{pmatrix} 1 & 1 \\ 1 & 1.0001 \end{pmatrix}, \quad \mathbf{b} = \begin{pmatrix} 2 \\ 2.0001 \end{pmatrix}.$$

Compute the condition number of $\mathbf{A}$ using np.linalg.cond.
Solve for $\mathbf{x}$ and perturb $\mathbf{b}$ by a small amount (e.g., add 0.001 to each element). Observe the change in $\mathbf{x}$.
Scale the rows of $\mathbf{A}$ and $\mathbf{b}$ to have unit norm (e.g., divide each row by its Euclidean norm) and repeat the process. Compare the condition numbers and solution sensitivities.

Hints: Use np.linalg.solve for solving. Discuss why scaling helps in ill-conditioned systems.

In [3]:
import numpy as np

# Define A and b
A = np.array([[1, 1], [1, 1.0001]])
b = np.array([2, 2.0001])

# Compute condition number
cond = np.linalg.cond(A)
print(f"Condition number: {cond}")

# Solve original system
x = np.linalg.solve(A, b)

# Perturb b and solve again
b_perturbed = b + 0.001
x_perturbed = np.linalg.solve(A, b_perturbed)
print(f"Relative change in x: {np.linalg.norm(x - x_perturbed) / np.linalg.norm(x)}")

# Scaling: Normalize rows
row_norms = np.linalg.norm(A, axis=1)
A_scaled = A / row_norms[:, np.newaxis]
b_scaled = b / row_norms

# Repeat condition and solves...

Condition number: 40002.000074915224
Relative change in x: 0.0007071067811864696


## Excercise 6: Implementing and Comparing Gaussian Elimination vs. LU Decomposition

In the excercise we want to demonstrate the relationship between Gaussian elimination (which you might remember from undergrad linear algebra classes) and LU decomposition, implement both from scratch, and compare their efficiency and stability for solving linear systems.

Consider the linear system $\mathbf{Ax} = \mathbf{b}$ with:

$$\mathbf{A} = \begin{pmatrix} 2 & 1 & 1 \\ 4 & -6 & 0 \\ -2 & 7 & 2 \end{pmatrix}, \quad \mathbf{b} = \begin{pmatrix} 5 \\ -2 \\ 9 \end{pmatrix}.$$

1.Implement Gaussian elimination without pivoting to solve for $\mathbf{x}$. Track the elimination steps and compute the effective upper triangular matrix.
2. Implement LU decomposition without pivoting (i.e., factor $\mathbf{A} = \mathbf{LU}$, where $\mathbf{L}$ has 1s on the diagonal).
3. Solve the system using the LU factors (forward substitution for $\mathbf{Ly} = \mathbf{b}$, then back substitution for $\mathbf{Ux} = \mathbf{y}$).
4. Compare the results from both methods. Then, test with multiple $\mathbf{b}$ vectors (e.g., add $\mathbf{b}_2 = [1, 2, 3]^\top$) and measure computation time using %timeit in Jupyter.
5. Introduce a small perturbation to $\mathbf{A}$ (e.g., add 1e-10 to one element) and discuss stability differences. Optionally, add partial pivoting to both implementations and recompare.

Hints:

1. In Gaussian elimination, augment $\mathbf{A}$ with $\mathbf{b}$ and perform row operations.
1. For LU, store multipliers in $\mathbf{L}$ below the diagonal during elimination.
1. Use NumPy for array operations but avoid built-in solvers like np.linalg.solve—implement the algorithms manually for learning.
1. Reference SciPy's scipy.linalg.lu_factor for validation, but explain why it's more robust.

Both methods should yield $\mathbf{x} \approx [1, 2, 2]^\top$. LU is more efficient for multiple $\mathbf{b}$ since factorization is done once. Without pivoting, both can fail on singular/ill-conditioned matrices—discuss how partial pivoting (e.g., swapping rows) improves stability, mirroring real-world implementations like `scipy.linalg.lu`.

The full algorithms are provided below to get you started:

#### Gaussian Elimination Algorithm
Gaussian elimination transforms the augmented matrix $[\mathbf{A} | \mathbf{b}]$ into row echelon form (upper triangular) through row operations, then solves via back substitution. For a system $\mathbf{Ax} = \mathbf{b}$ where $\mathbf{A} \in \mathbb{R}^{n \times n}$, the algorithm proceeds as follows (without pivoting for simplicity):

1. Forward Elimination Phase:
1.1 For each pivot row $k = 1$ to $n-1$:
For each row $i = k+1$ to $n$:
Compute the multiplier: $m_{ik} = \frac{a_{ik}}{a_{kk}}$
Update row $i$: $a_{ij} \leftarrow a_{ij} - m_{ik} a_{kj}$ for $j = k$ to $n$
Update right-hand side: $b_i \leftarrow b_i - m_{ik} b_k$

This results in an upper triangular matrix $\mathbf{U}$ and updated $\mathbf{b}'$.Symbolically, the elimination step for element $a_{ij}$ (with $i > k, j \geq k$) is:$$a_{ij}^{(k)} = a_{ij}^{(k-1)} - \frac{a_{ik}^{(k-1)}}{a_{kk}^{(k-1)}} a_{kj}^{(k-1)}$$where superscript $(k)$ denotes after the $k$-th elimination step.
1.2 Back Substitution Phase:
Starting from the last equation:$$x_n = \frac{b_n'}{u_{nn}}$$Then for $i = n-1$ down to $1$:$$x_i = \frac{b_i' - \sum_{j=i+1}^n u_{ij} x_j}{u_{ii}}$$

This method has a time complexity of $O(n^3)$ due to the nested loops in elimination.

2. LU Decomposition Algorithm
LU decomposition factors $\mathbf{A}$ into a lower triangular matrix $\mathbf{L}$ (with 1s on the diagonal) and an upper triangular matrix $\mathbf{U}$, such that $\mathbf{A} = \mathbf{LU}$. This is equivalent to Gaussian elimination but stores the multipliers in $\mathbf{L}$. For solving $\mathbf{Ax} = \mathbf{b}$, first solve $\mathbf{Ly} = \mathbf{b}$ (forward substitution), then $\mathbf{Ux} = \mathbf{y}$ (back substitution).

2.1 Factorization Phase (Doolittle's Algorithm, without pivoting):
Initialize $\mathbf{L} = \mathbf{I}$ (identity) and $\mathbf{U} = \mathbf{A}$.For each column $k = 1$ to $n$:
For the upper part ($j = k$ to $n$): $u_{kj} = a_{kj}$ (already set)
For the multipliers ($i = k+1$ to $n$):
$$l_{ik} = \frac{u_{ik}}{u_{kk}}, \quad u_{ij} \leftarrow u_{ij} - l_{ik} u_{kj} \quad \forall j \geq k$$
More formally, for $k = 1$ to $n$:$$u_{kk} = a_{kk} - \sum_{m=1}^{k-1} l_{km} u_{mk}$$For $j = k+1$ to $n$:$$u_{kj} = a_{kj} - \sum_{m=1}^{k-1} l_{km} u_{mj}$$For $i = k+1$ to $n$:$$l_{ik} = \frac{1}{u_{kk}} \left( a_{ik} - \sum_{m=1}^{k-1} l_{im} u_{mk} \right)$$
2.2 Forward Substitution ($\mathbf{Ly} = \mathbf{b}$):$$y_1 = \frac{b_1}{l_{11}} = b_1 \quad (\text{since } l_{11} = 1)$$For $i = 2$ to $n$:$$y_i = b_i - \sum_{j=1}^{i-1} l_{ij} y_j$$
2.3 Back Substitution ($\mathbf{Ux} = \mathbf{y}$):
Identical to Gaussian elimination's back substitution phase.

LU decomposition also has $O(n^3)$ complexity for factorization but allows efficient reuse for multiple $\mathbf{b}$ vectors (each solve is $O(n^2)$).
Note on Pivoting: In practice, partial pivoting is added to both methods for numerical stability, swapping rows to select the largest pivot and avoid division by small numbers. This modifies the algorithms to produce $\mathbf{PA} = \mathbf{LU}$ where $\mathbf{P}$ is a permutation matrix.

### Exercise 7: Comparing Direct Linear Solvers (LU vs. SVD)
In this exertcise we evaluate stability and performance of direct solvers for ill-conditioned systems.
Problem: Generate a Hilbert matrix $\mathbf{H}_n$ (ill-conditioned) of size $n=10$:
$$H_{ij} = \frac{1}{i + j - 1}$$

Solve $\mathbf{Hx} = \mathbf{b}$ (let $\mathbf{b} = \mathbf{H} \cdot \mathbf{1}$, true $\mathbf{x} = \mathbf{1}$) using LU decomposition (scipy.linalg.lu_solve) and SVD (np.linalg.pinv).
Perturb $\mathbf{b}$ slightly and compare recovered $\mathbf{x}$ errors.
Measure computation time for increasing $n$.

Hints: Use scipy.linalg.lu_factor for LU. SVD is more robust for rank-deficient cases.

### Exercise 8: Implementing and Testing Gauss-Seidel Iteration

Objective: Implement an iterative solver and analyze convergence conditions.
Problem: Solve the system from the chapter example:
$$\mathbf{A} = \begin{pmatrix} 4 & -1 & 0 \\ -1 & 4 & -1 \\ 0 & -1 & 4 \end{pmatrix}, \quad \mathbf{b} = \begin{pmatrix} 2 \\ 3 \\ 2 \end{pmatrix}.$$

Implement Gauss-Seidel with a tolerance of 1e-6 and max 100 iterations.
Test on a non-diagonally dominant matrix (e.g., swap rows) and observe divergence.
Add SOR with $\omega = 1.2$ and compare iteration counts.

Hints: Check diagonal dominance: $|a_{ii}| \geq \sum_{j \neq i} |a_{ij}|$.